In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime
import copy
import json
from glob import glob
import time
import os
from os.path import join
import pickle as pkl

import sys
sys.path.append('../../')

from data.processing import get_data

from utils.fitting.loss import Loss_Calculator
from viz import plot_backtest_seir

# Jul

In [ ]:
output_folder = '../../misc/reports/{}'.format("1594480618.481472")

In [ ]:
filename = '../../misc/reports/1594480618.481472/output-1594480618.481472.csv'

In [ ]:
df = pd.read_csv(filename)
cols_to_delete = [x for x in df.columns if ('max' in x) or ('min' in x)]
df = df.drop(cols_to_delete, axis=1)
df = df.drop(['current_active', 'current_icu', 'current_ventilator', 'icu_mean', 'hospitalized_mean'], axis=1)
df.rename({'current_hospitalized':'current_active'}, axis=1, inplace=True)
df.columns = [x if 'current' not in x else x.replace('current', 'true') for x in df.columns]
df.columns = [x if 'mean' not in x else 'pred_'+x.replace('_mean', '') for x in df.columns]
df.columns

In [ ]:
df_decile = df[df['which_forecast'] == '80.0']
df_decile

# Aug

In [ ]:
filename = '../../misc/oncall/08_2020_aug/nayana'

In [ ]:
df = pd.read_csv(glob(f'{filename}/*')[0])

In [ ]:
df['Date'x]

# Sept

In [ ]:
with open(join(output_folder, 'predictions_dict.pkl'), 'rb') as f:
    predictions_dict = pkl.load(f)

In [ ]:
fig = plot_backtest_seir(predictions_dict, df_true, which_forecast=50, truncate_pretrain_data=True, separate_compartments=False)

In [ ]:
fig = plot_backtest_seir(predictions_dict, df_true, which_forecast=50, truncate_pretrain_data=True, separate_compartments=True)

In [ ]:
lc = Loss_Calculator()
forecast_errors_dict = {}
df_train = predictions_dict['m2']['df_train']

for key, df_prediction in predictions_dict['m2']['forecasts'].items():
    if key in ['best', 50, 80]:
        df_prediction = copy.copy(df_prediction.loc[(df_prediction['date'] > df_train.iloc[-1, :]['date']) & 
                                                    (df_prediction['date'] <= df_true.iloc[-1, :]['date'])])
        df_true = df_true.loc[df_true['date'] > df_train.iloc[-1, :]['date']]
        df_prediction.reset_index(inplace=True, drop=True)
        df_true.reset_index(inplace=True, drop=True)
        ld = lc.calc_loss_dict(df_prediction, df_true, method='mape')
        ld = {key : round(value, 2) for key, value in ld.items()}
        forecast_errors_dict[key] = ld

In [ ]:
pd.DataFrame.from_dict(forecast_errors_dict)

In [ ]:
lc = Loss_Calculator()
forecast_errors_dict = {}
df_train = predictions_dict['m2']['df_train']

for key, df_prediction in predictions_dict['m2']['forecasts'].items():
    if key in [80]:
        df_prediction = copy.copy(df_prediction.loc[(df_prediction['date'] > df_train.iloc[-1, :]['date']) & 
                                                    (df_prediction['date'] <= df_true.iloc[-1, :]['date'])])
        df_true = copy.copy(df_true.loc[df_true['date'] > df_train.iloc[-1, :]['date']])
        df_prediction.reset_index(inplace=True, drop=True)
        df_true.reset_index(inplace=True, drop=True)
        week_indices = np.concatenate((np.arange(0, len(df_true), 7), [len(df_true)]))
        for i in range(len(week_indices)-1):
            df_prediction_slice = df_prediction.loc[week_indices[i]:week_indices[i+1]-1, :]
            df_true_slice = df_true.loc[week_indices[i]:week_indices[i+1]-1, :]
        
            ld = lc.calc_loss_dict(df_prediction_slice, df_true_slice, method='mape')
            
            ld = {key : round(value, 2) for key, value in ld.items()}
            forecast_errors_dict[f'week {i+1}'] = ld
            
        ld = lc.calc_loss_dict(df_prediction, df_true, method='mape')
            
        ld = {key : round(value, 2) for key, value in ld.items()}
        forecast_errors_dict['total'] = ld

In [ ]:
df = pd.DataFrame.from_dict(forecast_errors_dict)
df.index = ['active', 'recovered', 'deceased', 'total']
df

In [ ]:
predictions_dict['m2']['forecasts'][80]